In [1]:
import pandas as pd

In [5]:
df = pd.read_csv("../data/nfe_produto_1000_estruturado_anotado_test.csv")
df.head()

,text
0,"ARROZ PARBOLIZADO,LONGO FINO T1,PCT 1.0KG,BLUE..."
1,BISCOITO MAISENA C/ 400G - MABEL
2,FILé DE MERLUZA CONG.
3,AçúCAR CRISTAL PINDORAMA FD30X1KG
4,MINGAU DE MILHO VERDE COM CòCO


In [6]:
df.iloc[10,0]

'4957 - FLOCãO DE MILHO'

In [19]:
# This code is for v1 of the openai package: pypi.org/project/openai
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

client = OpenAI(api_key=openai_api_key)

response = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {
      "role": "system",
      "content": "Você é um assistente que organiza itens de notas fiscais. "
    },
    {
      "role": "user",
      "content": "Para cada descrição oferecida, organize os dados em formato JSON, nos campos \"produto\", \"marca\", \"quantidade\", \"unidade\", \"apresentacao\" e \"observacao\".\nComplete o nome do produto se estiver abreviado, utilizando a primeira letra maiúscula seguida de minúsculas. Remova quaisquer adjetivos ou descrições adicionais que não façam parte do nome do produto.\nA marca deve ser preenchida apenas quando uma segunda entidade for mencionada de forma que possa ser interpretada como tal; caso contrário, deixe o campo \"marca\" vazio (\"\").\nA \"quantidade\" deve refletir o peso total ou o conteúdo total de um único item ou pacote.\nA \"unidade\" deve estar na unidade do SI correspondente (g, kg, etc.), quando aplicável.\nSe a descrição mencionar um número de caixas ou pacotes, detalhe isso no campo \"apresentacao\" de forma apropriada (ex: \"pacote de 200g\", \"garrafa de 2l\").\nCorrija abreviações de unidades de medida para suas formas corretas no SI (\"GR\", \"GRS\" para \"g\"; \"KGS\", \"KLS\" para \"kg\").\nConverta números com vírgula para decimais em formato americano (com ponto).\nSe um campo não puder ser preenchido com as informações fornecidas, use aspas duplas (\"\") para indicar a ausência de dados.\nO campo \"observacao\" deve ser usado para qualquer informação adicional que não se encaixe nos outros campos.\nNão adicione campos extras ao JSON. Qualquer informação adicional deve ser incluída no campo \"observacao\".\n \n### Descrição:\nARROZ POLIDO AGULHINHA TIPO 01 REI DA PANELA 5KG\n \n### Resposta:\n{row[\"estruturado\"]}\n \n### Fim"
    },
    {
      "role": "assistant",
    #   "content": "{\n\"produto\": \"Arroz polido agulhinha tipo 01\",\n\"marca\": \"Rei da Panela\",\n\"quantidade\": \"5\",\n\"unidade\": \"kg\",\n\"apresentacao\": \"Pacote de 5kg\",\n\"observacao\": \"\"\n}"
      "content": df.iloc[10,0]
    }
  ],
  temperature=0,
  max_tokens=600,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [20]:
print(response.choices[0].message.content)

{ "produto": "Floco de milho", "marca": "", "quantidade": "", "unidade": "", "apresentacao": "", "observacao": "" }


In [41]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

def nfe_parser_gpt(df):
    client = OpenAI(api_key=openai_api_key)
    
    response_list = []
    for item in df['text']:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {
                "role": "system",
                "content": "Você é um assistente que organiza itens de notas fiscais. "
                },
                {
                "role": "user",
                "content": "Para cada descrição oferecida, organize os dados em formato JSON, nos campos \"produto\", \"marca\", \"quantidade\", \"unidade\", \"apresentacao\" e \"observacao\".\nComplete o nome do produto se estiver abreviado, utilizando a primeira letra maiúscula seguida de minúsculas. Remova quaisquer adjetivos ou descrições adicionais que não façam parte do nome do produto.\nA marca deve ser preenchida apenas quando uma segunda entidade for mencionada de forma que possa ser interpretada como tal; caso contrário, deixe o campo \"marca\" vazio (\"\").\nA \"quantidade\" deve refletir o peso total ou o conteúdo total de um único item ou pacote.\nA \"unidade\" deve estar na unidade do SI correspondente (g, kg, etc.), quando aplicável.\nSe a descrição mencionar um número de caixas ou pacotes, detalhe isso no campo \"apresentacao\" de forma apropriada (ex: \"pacote de 200g\", \"garrafa de 2l\").\nCorrija abreviações de unidades de medida para suas formas corretas no SI (\"GR\", \"GRS\" para \"g\"; \"KGS\", \"KLS\" para \"kg\").\nConverta números com vírgula para decimais em formato americano (com ponto).\nSe um campo não puder ser preenchido com as informações fornecidas, use aspas duplas (\"\") para indicar a ausência de dados.\nO campo \"observacao\" deve ser usado para qualquer informação adicional que não se encaixe nos outros campos.\nNão adicione campos extras ao JSON. Qualquer informação adicional deve ser incluída no campo \"observacao\".\n \n### Descrição:\nARROZ POLIDO AGULHINHA TIPO 01 REI DA PANELA 5KG\n \n### Resposta:\n{row[\"estruturado\"]}\n \n### Fim"
                },
                {
                "role": "assistant",
                #   "content": "{\n\"produto\": \"Arroz polido agulhinha tipo 01\",\n\"marca\": \"Rei da Panela\",\n\"quantidade\": \"5\",\n\"unidade\": \"kg\",\n\"apresentacao\": \"Pacote de 5kg\",\n\"observacao\": \"\"\n}"
                "content": item
                }
            ],
            temperature=0,
            max_tokens=600,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        response_list.append(response.choices[0].message.content)
    return response_list

In [42]:
df_menor = df.head(3)
response_list = nfe_parser_gpt(df_menor)

In [43]:
len(response_list)

3

In [48]:
print(response_list)

['{\n"produto": "Arroz parbolizado",\n"marca": "Blue Vile",\n"quantidade": "1.0",\n"unidade": "kg",\n"apresentacao": "Pacote 1.0kg",\n"observacao": "Longo fino tipo 1"\n}', '{\n"produto": "Biscoito Maizena",\n"marca": "Mabel",\n"quantidade": "400",\n"unidade": "g",\n"apresentacao": "Pacote de 400g",\n"observacao": ""\n}', '{ "produto": "Filé de Merluza", "marca": "", "quantidade": "", "unidade": "", "apresentacao": "Congelado", "observacao": "" }']


In [44]:
for item in response_list:
    print(item)
    print('------------------------')

{
"produto": "Arroz parbolizado",
"marca": "Blue Vile",
"quantidade": "1.0",
"unidade": "kg",
"apresentacao": "Pacote 1.0kg",
"observacao": "Longo fino tipo 1"
}
------------------------
{
"produto": "Biscoito Maizena",
"marca": "Mabel",
"quantidade": "400",
"unidade": "g",
"apresentacao": "Pacote de 400g",
"observacao": ""
}
------------------------
{ "produto": "Filé de Merluza", "marca": "", "quantidade": "", "unidade": "", "apresentacao": "Congelado", "observacao": "" }
------------------------


In [49]:
# Convertendo as strings para dicionários usando json.loads
import json
list_of_dicts = [json.loads(d) for d in response_list]
list_of_dicts


[{'produto': 'Arroz parbolizado',
  'marca': 'Blue Vile',
  'quantidade': '1.0',
  'unidade': 'kg',
  'apresentacao': 'Pacote 1.0kg',
  'observacao': 'Longo fino tipo 1'},
 {'produto': 'Biscoito Maizena',
  'marca': 'Mabel',
  'quantidade': '400',
  'unidade': 'g',
  'apresentacao': 'Pacote de 400g',
  'observacao': ''},
 {'produto': 'Filé de Merluza',
  'marca': '',
  'quantidade': '',
  'unidade': '',
  'apresentacao': 'Congelado',
  'observacao': ''}]

In [50]:
# Cria o Dataframe a partir da lista de dicionários
df_response = pd.DataFrame(list_of_dicts)
df_response

,produto,marca,quantidade,unidade,apresentacao,observacao
0,Arroz parbolizado,Blue Vile,1.0,kg,Pacote 1.0kg,Longo fino tipo 1
1,Biscoito Maizena,Mabel,400,g,Pacote de 400g,
2,Filé de Merluza,,,,Congelado,


In [52]:
# Salva o dataframe em um arquivo parquet
df_response.to_parquet('../data/nfe_produto_1000_estruturado_anotado_test.parquet')

In [ ]:
# Testando a leitura do arquivo parquet
df_from_parquet = pd.read_parquet('../data/nfe_produto_1000_estruturado_anotado_test.parquet')
df_from_parquet.head()